# Проект по SQL

# Описание проекта.
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Задачи проекта:
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## 1. Выгрузим данные

In [2]:
import pandas as pd
from sqlalchemy import create_engine 

In [3]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

In [4]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## 2. Исследуем имеющиеся таблицы.

In [5]:
books='''
SELECT *
FROM books LIMIT(10);
'''

pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


**Исследуем таблицы.**

In [6]:
authors='''
SELECT *
FROM authors LIMIT(10);
'''
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [7]:
publishers='''
SELECT *
FROM publishers LIMIT(10);
'''
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


In [8]:
ratings='''
SELECT *
FROM ratings LIMIT(10);
'''
pd.io.sql.read_sql(ratings, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [9]:
reviews='''
SELECT *
FROM reviews LIMIT(10);
'''
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


## 3. Посчитаем, сколько книг вышло после 1 января 2000 года.

In [10]:
count_books='''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >'2000-01-01';
'''
pd.io.sql.read_sql(count_books, con = engine)

,count
0,819


**Комментарий.** После 1 января 2000 года выпустилось 819 книг.

## 4. Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [11]:
books_rating= '''
SELECT b.title,
b.book_id,
COUNT(DISTINCT re.text)AS text_count,
AVG(r.rating) AS avg_rating
FROM books b
LEFT JOIN ratings r ON  b.book_id=r.book_id
LEFT JOIN reviews re ON re.book_id=b.book_id
GROUP BY b.title,
b.book_id 
ORDER BY text_count DESC

'''
pd.io.sql.read_sql(books_rating, con = engine)

,title,book_id,text_count,avg_rating
0,Twilight (Twilight #1),948,7,3.662500
1,Water for Elephants,963,6,3.977273
2,The Glass Castle,734,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
996,The Natural Way to Draw,808,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
998,Essential Tales and Poems,221,0,4.000000


**Комментарий**  Самое большое количесво обзоров у книги Twilight (Twilight #1)	, средняя оценка данной книги составила (3.66), также много книг у которых количнссво обзоров 6. Самая высокая средняя оценка около 5.

## 5. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
publisher='''
SELECT publisher,
COUNT(b.book_id) AS book_count
FROM publishers p
INNER JOIN books b ON p.publisher_id=b.publisher_id
WHERE num_pages> 50
GROUP BY p.publisher
ORDER BY book_count DESC
LIMIT(10);

'''
pd.io.sql.read_sql(publisher, con = engine)

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


**Комментарий** Издательство Penguin Books выпустило максимальное количесво книг(42), толще 50 страниц.

## 6. Определим автора с самой высокой средней оценкой книг — учитём только книги с 50 и более оценками.

In [20]:
autor= '''
SELECT
      p.author AS author,
      AVG(p.avg_rating) AS avg_rating
FROM
(SELECT
     a.author AS author,
     b.book_id,
     AVG(r.rating) AS avg_rating
FROM
    authors a
INNER JOIN books b ON b.author_id = a.author_id
INNER JOIN ratings r ON r.book_id = b.book_id
GROUP BY
    author,
    b.book_id
    HAVING
    COUNT(r.rating) >= 50) AS p
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''

pd.io.sql.read_sql(autor, con = engine)



,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


**Комментарий.** Самая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré 4.28.

## 7. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
avg_text='''
SELECT ROUND(avg(count)) 
FROM
(SELECT
    COUNT(text)
FROM 
    reviews
FULL OUTER JOIN (SELECT username,
                COUNT(rating_id) as count_r
FROM 
    ratings
GROUP BY
        username) AS count_r
ON count_r.username = reviews.username
where count_r  > 50
GROUP BY reviews.username) as count;
'''
pd.io.sql.read_sql(avg_text, con = engine)

,round
0,24.0


# 8. Bывод:  
В таблице count_books лежит информация о количесве книг вышедших после 1 января 2000 года, таких книг выпустилось 819.

В таблице books_rating лежит информация о средних оценках и количесве обзорах на книги. Самая высокая средняя оценка около 5, количесво большое число количесво обзоров составило 7.

Также нами был найден автор у которго самая высока средняя оценка  J.K. Rowling/Mary GrandPré 4.28 эти данные хранятся в таблице autor.

Мы посчитали  среднее количество обзоров от пользователей, которые поставили больше 50 оценок. оно составило 24 эти данные хоанятся в таблице avg_text.

Было найдено издательство которое выпустило наибольшее число книг толще 50 страниц-Издательство Penguin Books выпустило максимальное количесво книг(42). Эти данные хранятся в таблице publisher.